In [28]:
import sys
import cv2 as cv
import numpy as np

def hough(filename):
    # Loads an image
    src = cv.imread(cv.samples.findFile(filename), cv.IMREAD_COLOR)
    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    gray = cv.medianBlur(gray, 5)

    rows = gray.shape[0]
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, rows / 25,
                              param1=45, param2=30,
                              minRadius=1, maxRadius=30)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        r_list, c_list = [], []
        for i in circles[0, :]:
            center = (i[0], i[1])
            # circle center
            cv.circle(src, center, 1, (0, 100, 100), 3)
            # circle outline
            radius = i[2]
            cv.circle(src, center, radius, (255, 0, 255), 3)
            r_list.append(radius)
            c_list.append(center)
            
    print(len(r_list))
    print(r_list)
    print(c_list)

    #cv.imshow("detected circles", src)
    #cv.waitKey(0)

In [32]:
import os
import pandas as pd

folder = '/Users/Zack/0_thesis_bing/detections_1+/detections/'
images = os.listdir(os.path.join(folder))

object_list = pd.DataFrame([])
for i in images[0:5]:
    filename = folder + i
    print(i)
    hough(filename)
    objects = pd.DataFrame(data = {'radius':r_list, 'center':c_list} ) #create df of detections
    objects['image_name'] = i #add image name
    
    #combine dections from all images
    object_list = object_list.append(pd.DataFrame(objects), sort=False) 
object_list

image_(18, 35.29339, 26.26795, 40.53992, 22.75128, 40.54879, 22.73931).jpeg
140
[29, 29, 29, 29, 27, 29, 29, 29, 29, 29, 29, 27, 27, 24, 27, 24, 24, 27, 27, 27, 27, 22, 24, 24, 29, 24, 27, 27, 24, 24, 24, 22, 22, 22, 22, 22, 22, 24, 17, 22, 22, 22, 17, 25, 19, 17, 24, 22, 22, 19, 17, 17, 20, 20, 19, 19, 17, 17, 17, 17, 22, 17, 17, 22, 20, 17, 22, 17, 17, 17, 19, 16, 17, 17, 17, 17, 22, 17, 17, 16, 24, 19, 17, 17, 15, 15, 17, 17, 16, 16, 14, 17, 17, 17, 17, 16, 15, 15, 15, 14, 14, 17, 15, 14, 12, 15, 14, 13, 17, 16, 19, 15, 14, 14, 13, 15, 13, 14, 14, 15, 14, 14, 13, 12, 12, 8, 16, 12, 10, 16, 14, 12, 10, 13, 12, 12, 10, 8, 8, 12]
[(1922, 288), (2052, 542), (1198, 1972), (1168, 112), (2124, 80), (1844, 864), (1320, 482), (1556, 172), (1646, 238), (1862, 694), (1780, 1390), (2110, 884), (2130, 1638), (778, 572), (2082, 1278), (1502, 542), (1910, 1728), (1092, 1856), (1510, 2076), (1932, 444), (2172, 1714), (1492, 344), (1852, 364), (2020, 952), (966, 2104), (2098, 1018), (1792, 272), (10

278
[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 27, 27, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 28, 27, 24, 27, 24, 24, 27, 27, 26, 29, 28, 27, 24, 24, 29, 24, 24, 22, 22, 27, 27, 27, 24, 22, 27, 27, 25, 22, 22, 25, 24, 24, 22, 22, 22, 22, 27, 24, 24, 22, 24, 24, 22, 19, 24, 24, 24, 22, 22, 19, 27, 24, 24, 23, 22, 22, 22, 22, 24, 22, 22, 22, 22, 22, 22, 22, 20, 19, 19, 19, 24, 24, 22, 22, 22, 22, 19, 22, 22, 22, 22, 22, 19, 17, 24, 22, 20, 19, 19, 19, 19, 17, 22, 22, 22, 20, 19, 19, 19, 17, 24, 22, 22, 22, 20, 19, 19, 19, 19, 19, 17, 17, 20, 19, 19, 19, 19, 19, 19, 18, 17, 17, 20, 19, 19, 19, 19, 19, 17, 20, 19, 19, 18, 17, 17, 17, 17, 22, 19, 18, 17, 17, 17, 17, 17, 17, 17, 18, 17, 17, 17, 17, 16, 19, 19, 17, 17, 17, 16, 13, 17, 17, 17, 18, 18, 17, 17, 17, 17, 17, 18, 17, 17, 17, 17, 13, 12, 17, 17, 17, 17, 17, 17, 16, 15, 15, 15, 14, 14, 14, 14, 17, 15, 15, 15, 14, 14, 18, 17, 14, 14, 16, 16, 16, 15, 14, 14, 16, 15, 14, 15, 15, 15, 14, 14, 12, 12, 14, 14, 14, 13, 14,

,radius,center,image_name
0,17,"(828, 840)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
1,22,"(438, 1346)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
2,21,"(522, 886)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
3,21,"(694, 822)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
4,21,"(418, 986)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
5,21,"(400, 1242)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
6,21,"(306, 1024)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
7,21,"(650, 902)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
8,20,"(560, 1162)","image_(18, 35.29339, 26.26795, 40.53992, 22.75..."
0,17,"(828, 840)","image_(18, 35.29715, 25.47558, 40.50499, 22.70..."


In [33]:
object_list.shape

(45, 3)

In [34]:
objects

,radius,center,image_name
0,17,"(828, 840)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
1,22,"(438, 1346)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
2,21,"(522, 886)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
3,21,"(694, 822)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
4,21,"(418, 986)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
5,21,"(400, 1242)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
6,21,"(306, 1024)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
7,21,"(650, 902)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
8,20,"(560, 1162)","image_(18, 35.30221, 26.29449, 40.54475, 22.77..."
